# Notes:

make altitude a small array and check tht its continues

In [1]:
filename = '../data/TIEGCM_CCMC/my_stuff/s_002.nc'

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd
import numpy as np
from scipy.spatial import kdtree
from scipy.interpolate import griddata, LinearNDInterpolator
import scipy
from netCDF4 import Dataset
from plotly.offline import plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import Layout
import plotly
import matplotlib.pyplot as plt
import antlr4 

plotly.offline.init_notebook_mode(connected=True)

In [4]:
from tiegcm.tiegcm import TIEGCM, Point3D, Point4D, Slice_key4D, ColumnSlice4D, ColumnSlice3D
from tiegcm.util import average_longitude
from tiegcm.util import *

from tiegcm.tiegcm_original import TIEGCM as TIEGCM_o
from tiegcm.tiegcm_original import Point3D as Point3D_o
from tiegcm.tiegcm_original import Point4D as Point4D_o
from tiegcm.tiegcm_original import Point4D as Point4D_o
from tiegcm.tiegcm_original import Slice_key4D as Slice_key4D_o
from tiegcm.tiegcm_original import ColumnSlice4D as ColumnSlice4D_o 
from tiegcm.tiegcm_original import ColumnSlice3D as ColumnSlice3D_o
from tiegcm.util_original import average_longitude as average_longitude_o
from tiegcm.util_original import *


In [5]:
tiegcm = TIEGCM(filename)  # Imports data file
tiegcm_o = TIEGCM_o(filename)  # Imports data file
None

initializing tiegcm with ../data/TIEGCM_CCMC/my_stuff/s_002.nc
initializing tiegcm with ../data/TIEGCM_CCMC/my_stuff/s_002.nc


In [6]:
# HE, O1, O2, N2, TotalDEN
# (time, lev, lat, lon)

lon = 20  # 72 total
lat = 34  # 36 total
time = 1


DEN = np.array(tiegcm.rootgrp.variables['DEN'])
ZG = np.array(tiegcm.rootgrp.variables['ZG'])

HE_mmr = np.array(tiegcm.rootgrp.variables['HE']) #fraction of mass that is helium
O1_mmr = np.array(tiegcm.rootgrp.variables['O1'])
O2_mmr = np.array(tiegcm.rootgrp.variables['O2'])
N2_mmr = np.array(tiegcm.rootgrp.variables['N2'])   # for the CCMC data this variable name is N2N


HE = HE_mmr*DEN/(1 + HE_mmr) #can't do this!
O1 = O1_mmr*DEN/(1 + O1_mmr)
O2 = O2_mmr*DEN/(1 + O2_mmr)
N2 = N2_mmr*DEN/(1 + N2_mmr)


# traceHE = go.Scatter(
#     x = HE[time, :, lat, lon],
#     y = 1e-5*ZG[time, :, lat, lon],
#     name='HE',
#     mode = 'lines+markers',
#     marker= dict(
#         size =4
#     )
#     )

# traceO1 = go.Scatter(
#     x = O1[time, :, lat, lon],
#     y = 1e-5*ZG[time, :, lat, lon],
#     name= 'O1',
#     mode = 'lines+markers',
#     marker= dict(
#         size =4
#     )
#     )

# traceO2 = go.Scatter(
#     x = O2[time, :, lat, lon],
#     y = 1e-5*ZG[time, :, lat, lon],
#     name='O2',
#     mode = 'lines+markers',
#     marker= dict(
#         size =4
#     )
#     )

# traceN2 = go.Scatter(
#     x = N2[time, :, lat, lon],
#     y = 1e-5*ZG[time, :, lat, lon],
#     name='N2',
#     mode = 'lines+markers',
#     marker= dict(
#         size =4
#     )
#     )

# traceDEN = go.Scatter(
#     x = DEN[time, :, lat, lon],
#     y = 1e-5*ZG[time, :, lat, lon],
#     name='Total Density',
#         line = dict(
#         color = ('rgb(10, 10, 10)'),
#         width = 2,
#         )  
# )

# data = [traceHE, traceO1, traceO2, traceN2, traceDEN]

# layout = go.Layout(
#         xaxis=dict(
#         type='log',
#         autorange=True,
#         exponentformat='power',
#         showexponent='all',
#         title = 'Density [kg/m^3]'
#         ),
#         yaxis=dict(
#         title = 'Altitude [km]'
#         ),
#         title = "Vertical Profile of Constituents from Original tiegcm.py"
# )

# fig = go.Figure(data=data, layout=layout)
# iplot(fig)
    
#iplot(data, filename='basic-line')

In [18]:
# HE, O1, O2, N2, TotalDEN
# (time, lev, lat, lon)

lon = 20  # 72 total
lat = 20  # 36 total
time = 1



DEN = 1000*np.array(tiegcm.rootgrp.variables['DEN'])   
ZG = np.array(tiegcm.rootgrp.variables['ZG'])
ZGMID = np.array(tiegcm.rootgrp.variables['ZGMID'])

HE_mmr = np.array(tiegcm.rootgrp.variables['HE'])
O1_mmr = np.array(tiegcm.rootgrp.variables['O1'])
O2_mmr = np.array(tiegcm.rootgrp.variables['O2'])
N2_mmr = np.array(tiegcm.rootgrp.variables['N2'])   # for the CCMC data this avriable name is N2N

HE = HE_mmr*DEN  #kg/m^3     #/(1 + HE_mmr)
O1 = O1_mmr*DEN  #kg/m^3     #/(1 + O1_mmr)
O2 = O2_mmr*DEN  #kg/m^3     #/(1 + O2_mmr)
N2 = N2_mmr*DEN  #kg/m^3     #/(1 + N2_mmr)



traceHE = go.Scatter(
    x = HE[time, :, lat, lon],
    y = 1e-5*ZGMID[time, :, lat, lon],
    name='HE ZGMID',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceHE_ZG = go.Scatter(
    x = HE[time, :, lat, lon],
    y = 1e-5*ZG[time, :, lat, lon],
    name='HE ZG',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceO1 = go.Scatter(
    x = O1[time, :, lat, lon],
    y = 1e-5*ZGMID[time, :, lat, lon],
    name= 'O1 ZGMID',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceO1_ZG = go.Scatter(
    x = O1[time, :, lat, lon],
    y = 1e-5*ZG[time, :, lat, lon],
    name= 'O1 ZG',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceO2 = go.Scatter(
    x = O2[time, :, lat, lon],
    y = 1e-5*ZGMID[time, :, lat, lon],
    name='O2 ZGMID',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceO2_ZG = go.Scatter(
    x = O2[time, :, lat, lon],
    y = 1e-5*ZG[time, :, lat, lon],
    name='O2 ZG',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceN2 = go.Scatter(
    x = N2[time, :, lat, lon],
    y = 1e-5*ZGMID[time, :, lat, lon],
    name='N2 ZGMID',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceN2_ZG = go.Scatter(
    x = N2[time, :, lat, lon],
    y = 1e-5*ZG[time, :, lat, lon],
    name='N2 ZG',
    mode = 'lines+markers',
    marker= dict(
        size =4
    )
    )
traceDEN = go.Scatter(
    x = DEN[time, :, lat, lon],
    y = 1e-5*ZG[time, :, lat, lon],
    name='Total Density (on ZG)',
        line = dict(
        color = ('rgb(0,0,255)'), 
        width = 2,
        )  
        )
trace_topPlevel = go.Scatter(
    x = [0, np.ndarray.max(DEN)],
    y = [1e-5*ZG[time, -2, lat, lon], 1e-5*ZG[time, -2, lat, lon]], 
    name = 'Top pressure level',
    mode = 'lines',
    line = dict(
        dash = 'dash',
        width = 1
        )
    )
trace_neg5thlevel = go.Scatter(
    x = [0, np.ndarray.max(DEN)],
    y = [1e-5*ZG[time, -4, lat, lon], 1e-5*ZG[time, -4, lat, lon]], 
    name = '4th from top pressure level',
    mode = 'lines',
    line = dict(
        dash = 'dash',
        width = 1
    )
    )

data = [traceHE, traceHE_ZG, 
        traceN2,
        traceO2,
        traceO1, traceO1_ZG, 
         traceO2_ZG, 
         traceN2_ZG, 
        traceDEN, trace_neg5thlevel]

layout = go.Layout(
        xaxis=dict(
        type='log',
        autorange=True,
        exponentformat='power',
        showexponent='all',
        title = 'Density [kg/m^3]'
        ),
        yaxis=dict(
        title = 'Altitude [km]'
        ),
        title = "Vertical Profile of Constituents on midpoint altitudes"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)
    


# Extrapolating Upwards!

In [8]:
lon = np.array(tiegcm.rootgrp.variables['lon'])
lat = np.array(tiegcm.rootgrp.variables['lat'])
lev = np.array(tiegcm.rootgrp.variables['ilev'])
ut = np.array(tiegcm.rootgrp.variables['ut'])


DEN = 1e3*np.array(tiegcm.rootgrp.variables['DEN']) # kg/m^3 
ZG = 1e-2*np.array(tiegcm.rootgrp.variables['ZG'])   # m
ZGMID = 1e-2*np.array(tiegcm.rootgrp.variables['ZGMID'])# m

HE_mmr = np.array(tiegcm.rootgrp.variables['HE']) # mass mixing ratio
O1_mmr = np.array(tiegcm.rootgrp.variables['O1']) # mass mixing ratio
O2_mmr = np.array(tiegcm.rootgrp.variables['O2']) # mass mixing ratio
N2_mmr = np.array(tiegcm.rootgrp.variables['N2']) # mmr  # for the CCMC data this avriable name is N2N

HE = HE_mmr*DEN#/(1 + HE_mmr)  #kg/m^3      #
O1 = O1_mmr*DEN#/(1 + HE_mmr)  #kg/m^3     #/(1 + O1_mmr)
O2 = O2_mmr*DEN#/(1 + HE_mmr)  #kg/m^3     #/(1 + O2_mmr)
N2 = N2_mmr*DEN#/(1 + HE_mmr)  #kg/m^3     #/(1 + N2_mmr)


user_lat = 16
user_lon = -4
user_ut = 12
user_lev = 25

# print HE_mmr/1+HE_mmr


In [9]:
R_e = 6.371008e6   #  [ m  ]
k = 1.38064852e-23 # [ J/K ] 
m_p = 1.6726e-27   # [  kg ]
g_0 = 9.807        # [m/s^2]

def gravity(h):
    """For h in same units as R_e"""
    return g_0*pow(R_e/(R_e + h), 2)

def scale_height(T, height, molecular_mass):
	'''Scale height in meters, assuming mks'''
	return k*T/(gravity(height)*molecular_mass*m_p)

In [10]:
zg_max = ZG[user_ut,:-4, user_lat, user_lon].max()  # we are going to use the 25 pressure level as the top boundary
zgmid_max = ZGMID[user_ut,:-4, user_lat, user_lon].max()  

altzg = np.arange(zg_max, 720000, 1000)    #ZG[user_ut, user_lev, user_lat, user_lon] #user defined altitude
altzgmid = np.arange(zgmid_max, 720000, 1000)    #ZG[user_ut, user_lev, user_lat, user_lon] #user defined altitude


# print altzgmid, 'km', '\n'
# print altzg, 'km', '\n'

print 'zg_max=',zg_max *1e-3
print 'zgmid_max=',zgmid_max *1e-3




zg_max= 450.34756250000004
zgmid_max= 465.01684375


In [11]:
# density_tot = tiegcm.rootgrp.variables['DEN'] # total mass density [g/cm^3]
# density_O1 = tiegcm.rootgrp.variables['O1'] 
# density_O2 = tiegcm.rootgrp.variables['O2'] 
# density_N2 = tiegcm.rootgrp.variables['N2'] 
# density_HE = tiegcm.rootgrp.variables['HE'] 

den_total = DEN[user_ut, :-4, user_lat, user_lon]
den_O1 = O1[user_ut, :-4, user_lat, user_lon]
den_O2 = O2[user_ut, :-4, user_lat, user_lon]
den_N2 = N2[user_ut, :-4, user_lat, user_lon]
den_HE = HE[user_ut, :-4, user_lat, user_lon]

denExtrap_O1 = []
denExtrap_O2 = []
denExtrap_HE = []
denExtrap_N2 = []
rho_alt = []

count = 0

for ialt in altzgmid:
    if ialt > zgmid_max.max():
        T = np.array(tiegcm.rootgrp.variables['TN']) # neutral temperature [K]
        T = T[user_ut, user_lev, user_lat, user_lon]

        h_O1 = scale_height(T, ialt, 16)
        denExtrap_O1.append((den_O1[-1])*np.exp((zgmid_max - ialt)/h_O1))

        h_O2 = scale_height(T, ialt, 32)
        denExtrap_O2.append((den_O2[-1])*np.exp((zgmid_max - ialt)/h_O2))

        h_HE = scale_height(T, ialt, 4)      # ZACH ADDED  
        denExtrap_HE.append((den_HE[-1])*np.exp((zgmid_max - ialt)/h_HE))    # ZACH ADDED      

        h_N2 = scale_height(T, ialt, 28)
        denExtrap_N2.append((den_N2[-1])*np.exp((zgmid_max - ialt)/h_N2))
    
        rho_alt.append(denExtrap_O1[count] + denExtrap_O2[count] + denExtrap_N2[count] + denExtrap_HE[count])
        count += 1 
        
        
print den_total
print 'rho above',rho_alt

    

[1.0614157e-06 6.9200047e-07 3.9340085e-07 2.0032276e-07 9.7152679e-08
 4.6450214e-08 2.2116366e-08 1.0772510e-08 5.5001457e-09 2.9265705e-09
 1.5892124e-09 8.7177499e-10 4.8324644e-10 2.7084179e-10 1.5325220e-10
 8.7372831e-11 5.0124242e-11 2.8922255e-11 1.6787976e-11 9.8054811e-12
 5.7626490e-12 3.4055291e-12 2.0213588e-12 1.2031423e-12 7.1681831e-13]
rho above [7.035216519373373e-13, 6.904853521699199e-13, 6.77704019505178e-13, 6.651723795006783e-13, 6.528852740249836e-13, 6.408376584987983e-13, 6.29024599206972e-13, 6.174412706793842e-13, 6.060829531388e-13, 5.949450300138393e-13, 5.840229855152579e-13, 5.733124022737965e-13, 5.628089590378998e-13, 5.525084284296642e-13, 5.424066747574146e-13, 5.324996518833642e-13, 5.227834011448555e-13, 5.132540493277192e-13, 5.039078066903418e-13, 4.947409650370604e-13, 4.857498958395576e-13, 4.769310484049556e-13, 4.682809480893558e-13, 4.597961945556026e-13, 4.514734600740839e-13, 4.4330948786541987e-13, 4.353010904839207e-13, 4.27445148240728

In [12]:
# height = 1e-5*ZG[user_ut, :, user_lat, user_lon]
# 1e-5*(alt-zg_max) 

# above_alt = np.linspace(1e-5*zg_max, 1e-5*alt)

# full_height =  np.append(height, above_alt)

In [13]:
# print den_HE.shape, ZGMID[user_ut, user_lev, user_lat, user_lon].shape

In [14]:
# x = np.arange(0,10)
# print x[0:-5]

In [15]:

trace_den_HE = go.Scatter(
    x = den_HE,
    y = 1e-3*ZGMID[user_ut, :-4, user_lat, user_lon],
    name= 'den_HE',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )

trace_den_O1 = go.Scatter(
    x = den_O1,
    y = 1e-3*ZGMID[user_ut, :-4, user_lat, user_lon],
    name='den_O1',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )


extrap_den_O1 = go.Scatter(
    x = denExtrap_O1,
    y = 1e-3*altzgmid,
    name='den_O1 above boundary',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )
extrap_den_O2 = go.Scatter(
    x = denExtrap_O2,
    y = 1e-3*altzgmid,
    name='den_O2 above boundary',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )
extrap_den_HE = go.Scatter(
    x = denExtrap_HE,
    y = 1e-3*altzgmid,
    name='den_HE above boundary',
    mode = 'lines+markers',
    marker= dict(
        #color = 'rgb(245,255,0)',
        size =2
    )
    )

extrap_den_N2= go.Scatter(
    x = denExtrap_N2,
    y = 1e-3*altzgmid,
    name='den_N2 above boundary',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )

trace_den_O2 = go.Scatter(
    x = den_O2,
    y = 1e-3*ZGMID[user_ut, :-4, user_lat, user_lon],
    name='den_O2',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )

trace_den_N2 = go.Scatter(
    x = den_N2,
    y = 1e-3*ZGMID[user_ut, :-4, user_lat, user_lon],
    name='den_N2',
    mode = 'lines+markers',
    marker= dict(
        size =2
    )
    )

trace_DenTot = go.Scatter(
    x = den_total,
    y = (1e-3*ZG[user_ut, :, user_lat, user_lon]),
    name='Mass Density from model',
        line = dict(
        width = 4,
        color = ('rgb(0,0,255)'),
        )  
        )

trace_Den_extrap = go.Scatter(
    x = rho_alt,
    y = 1e-3*altzgmid, #full_height,
    name='Extrapolated Mass Density above model',
        line = dict(
        width = 4,
        color = ('rgb(255, 0, 0)'),
        )
        )  


aqua_orbit = go.Scatter(
    x = [0, np.ndarray.max(DEN)],
    y = [703, 703], 
    name = 'Aqua Orbit altitude at 703 km',
    mode = 'lines',
    line = dict(
        dash = 'dash',
        width = 1
    )
    )

mplev25 = go.Scatter(
    x = [0, np.ndarray.max(DEN)],
    y = [1e-3*ZG[user_ut, -4, user_lat, user_lon], 1e-3*ZG[user_ut, -4, user_lat, user_lon]], 
    name = '4th from top pressure level',
    mode = 'lines',
    line = dict(
        dash = 'dash',
        width = 1
    )
    )


data = [trace_den_HE,trace_den_N2,trace_den_O2, trace_den_O1,
        extrap_den_HE,extrap_den_N2,extrap_den_O2, extrap_den_O1,    
        mplev25, aqua_orbit,
        trace_Den_extrap,trace_DenTot,]

layout = go.Layout(
        xaxis=dict(
        type='log',
        autorange=True,
        exponentformat='power',
        showexponent='all',
        title = 'Density [kg/m^3]'
        ),
        yaxis=dict(
        title = 'Altitude [km]'
        ),
        title = "Extrapolated mass densities above 25th lev"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename = 'latex')


In [16]:
def test_mass_density():
    tiegcm = TIEGCM_o(filename) 
    xlat = -37.5
    xlon = 100
    xalt = np.arange(100,700,1)*1e5 # cm
    time = 12 # ut hours
    vert_dens_profile = []
    
    for ixalt in xalt:
        test_dens = (tiegcm.density(xlat, xlon, ixalt, time))
        vert_dens_profile.append(test_dens)
        
        DEN = 1e3*np.array(tiegcm.rootgrp.variables['DEN'])
        HE_mmr = np.array(tiegcm.rootgrp.variables['HE'])
        HE = HE_mmr*DEN#/(1 + HE_mmr)  #kg/m^3      #
        user_lat = 10
        user_lon = -16
        user_ut = 12
        user_lev = 25
        den_HE = HE[user_ut, :-4, user_lat, user_lon]

        trace_den_HE = go.Scatter(
            x = den_HE,
            y = 1e-3*ZGMID[user_ut, :-4, user_lat, user_lon],
            name= 'den HE from netcdf',mode = 'lines+markers',marker= dict(size =2)) 
   
        trace_denCode = go.Scatter(
            x = 1e3*np.array(vert_dens_profile),
            y = 1e-5*xalt,
            name='density profile from reader',mode = 'markers',marker= dict(size =3) )
        
        data = [trace_denCode,trace_den_HE]

        layout = go.Layout(
            xaxis=dict(type='log', autorange=True, exponentformat='power',
                       showexponent='all',title = 'Density [kg/m^3]'    ),
            yaxis=dict(title = 'Altitude [km]'), title = "Mass Density, incl extrapolation")
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename = 'latex')

    return test_dens 

In [17]:
test_mass_density()

initializing tiegcm with ../data/TIEGCM_CCMC/my_stuff/s_002.nc


ValueError: operands could not be broadcast together with shapes (24,29,36,72) (24,29,38,73) 

In order to register midpoint quantities in altitude, it is therefore necessary to interpolate from the midpoints to the interfaces. Alternatively, it may be simpler to interpolate zg from the interfaces to the midpoints.
 (https://www.hao.ucar.edu/modeling/tgcm/tiegcm2.0/userguide/html/grid.html)

In [ ]:
# ilev = np.array(tiegcm.rootgrp.variables['ilev'])
# print ilev

# lev = np.array(tiegcm.rootgrp.variables['lev'])
# print lev

In [ ]:
# ZG = np.array(tiegcm.rootgrp.variables['ZG'])

# #use a variable that exists on midpoint (ie, O2)
# O2_midpoints = np.array(tiegcm.rootgrp.variables['O2'])  #this is on a midpoint
    
# #interpolate from the midpoints to the interfaces
# #it may be simpler to interpolate ZG from the interfaces to the midpoints

# lon = 20  # 72 total
# lat = 20  # 36 total
# time = 1  
    
# print O2_midpoints[time, :, lat, lon], '\n' #uses ilev  

# o2_interface = np.interp(ilev, O2_midpoints[time, :, lat, lon], O2_midpoints[time, :, lat, lon] )


# print o2_interpd